## Components and Residuals

* series = components + residuals<br>
&nbsp;(components: trend, seasons, cycles,   residuals: error)
1) first learn the trend and subtract it out from the series,
2) then learn the seasonality from the detrended residuals and subtract the seasons out,
3) then learn the cycles and subtract the cycles out,
4) and finally only the unpredictable error remains.
5) Add together all the components we learned and we get the complete model. (linear regression)



```python
# 1. Train and predict with first model
model_1.fit(X_train_1, y_train)
y_pred_1 = model_1.predict(X_train)

# 2. Train and predict with second model on residuals
model_2.fit(X_train_2, y_train - y_pred_1)
y_pred_2 = model_2.predict(X_train_2)

# 3. Add to get overall predictions
y_pred = y_pred_1 + y_pred_2
```

* Feature-transforming algorithms : linear regression, neural nets 
* Target-transforming algorithms : decision trees, nearest neighbors

* This difference is what motivates the hybrid design in this lesson: use linear regression to extrapolate the trend, transform the target to remove the trend, and apply XGBoost to the detrended residuals. To hybridize a neural net (a feature transformer), you could instead include the predictions of another model as a feature, which the neural net would then include as part of its own predictions. The method of fitting to residuals is actually the same method the gradient boosting algorithm uses, so we will call these boosted hybrids; the method of using predictions as features is known as "stacking", so we will call these stacked hybrids.